<a href="https://colab.research.google.com/github/GussAlves/Tech-Challenge---IA-para-Devs---Fase-2/blob/main/TFC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deap

In [ ]:
import random
import smtplib
import pandas as pd
import os
from cryptography.fernet import Fernet
from deap import base, creator, tools, algorithms
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

Parâmetros de entrada

In [ ]:
# Parâmetros de entrada
NUM_SENSORS = 10
NUM_AREAS = 5

CRITICITY_LEVELS = [4, 3, 5, 2, 1]  # Nível de criticidade de cada área (escala de 1 a 5)
PROB_ATTACK = [0.2, 0.1, 0.4, 0.05, 0.3]  # Probabilidade de um ataque ocorrer em cada área
COST = [600, 300, 900, 150, 500]  # Custo de proteção (instalação de sensores) em cada área
CAPACITY = [120, 80, 200, 60, 100]  # Capacidade de monitoramento dos sensores em cada área
VULNERABILITIES = [True, False, True, False, True]  # Indica se há vulnerabilidades conhecidas em cada área
CVSS_SCORES = [8.7, 0, 9.2, 0, 7.5]  # Pontuação CVSS (severidade da vulnerabilidade) para cada área
MAX_COST = 2500  # Custo máximo permitido para a instalação
MAX_CAPACITY = 400  # Capacidade máxima permitida de sensores

HISTORY_FILE = "historico_criticidade.txt"

In [ ]:
# Função para carregar a chave de criptografia
def load_encryption_key():
    with open("encryption_key.key", "rb") as key_file:
        return key_file.read()

# Função para descriptografar as credenciais
def load_encrypted_credentials():
    key = load_encryption_key()
    fernet = Fernet(key)

    with open("encrypted_credentials.txt", "rb") as file:
        encrypted_sender_email = file.readline().strip()
        encrypted_receiver_email = file.readline().strip()
        encrypted_password = file.readline().strip()

    sender_email = fernet.decrypt(encrypted_sender_email).decode()
    receiver_email = fernet.decrypt(encrypted_receiver_email).decode()
    password = fernet.decrypt(encrypted_password).decode()

    sender_email = "fiaptest@yahoo.com"
    receiver_email = "edurique69@gmail.com"
    password = "#Q@W#qw3Q@W#q2w3"

    return sender_email, receiver_email, password

# Função para enviar e-mail de alerta usando as credenciais descriptografadas
def send_email_alert(subject, body):
    sender_email, receiver_email, password = load_encrypted_credentials()

    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    # Criação da mensagem de e-mail
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))  # Adiciona o corpo do e-mail como texto simples

    try:
        # Conexão com o servidor SMTP
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Inicia a conexão criptografada TLS
        server.login(sender_email, password)  # Faz o login com o e-mail e senha

        # Envia o e-mail
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print("E-mail enviado com sucesso!")

        # Fecha a conexão com o servidor
        server.quit()
    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

    print("E-mail enviado com sucesso!")

# Função de fitness com avaliação de vulnerabilidade e CVSS
def eval_allocation(individual):
    total_critic = 0
    total_cost = 0
    total_capacity = 0

    for i in range(NUM_AREAS):
        if individual[i] == 1:
            vulnerability_factor = CVSS_SCORES[i] if VULNERABILITIES[i] else 1
            total_critic += CRITICITY_LEVELS[i] * PROB_ATTACK[i] * vulnerability_factor
            total_cost += COST[i]
            total_capacity += CAPACITY[i]

    if total_cost > MAX_COST or total_capacity > MAX_CAPACITY:
        return 0,
    return total_critic,

# Classificação de criticidade com base na pontuação CVSS
def classify_criticidade_cvss(cvss_score):
    if cvss_score >= 9.0:
        return "Crítica"
    elif 7.0 <= cvss_score < 9.0:
        return "Alta"
    elif 4.0 <= cvss_score < 7.0:
        return "Média"
    elif 0.1 <= cvss_score < 4.0:
        return "Baixa"
    else:
        return "Nenhuma"

# Função para gerar relatório
def generate_report(best_ind, best_fitness, criticidade, total_cost, total_capacity):
    areas = [f"Área {i+1}" for i in range(NUM_AREAS)]
    allocation = best_ind
    vulnerability_status = ["Vulnerável" if VULNERABILITIES[i] else "Seguro" for i in range(NUM_AREAS)]
    cvss_classification = [classify_criticidade_cvss(CVSS_SCORES[i]) for i in range(NUM_AREAS)]

    # Criação do DataFrame
    data = {
        "Área": areas,
        "Alocação de Sensores (1 = Sim, 0 = Não)": allocation,
        "Nível de Criticidade": CRITICITY_LEVELS,
        "Probabilidade de Ataque": PROB_ATTACK,
        "Custo de Implementação": COST,
        "Capacidade Utilizada": CAPACITY,
        "Vulnerabilidade por Falta de Patch": vulnerability_status,
        "Pontuação CVSS": CVSS_SCORES,
        "Classificação CVSS": cvss_classification
    }

    df = pd.DataFrame(data)

    # Informações gerais
    summary = {
        "Fitness": best_fitness,
        "Criticidade Global": criticidade,
        "Custo Total": total_cost,
        "Capacidade Total": total_capacity
    }

    summary_df = pd.DataFrame(list(summary.items()), columns=["Parâmetro", "Valor"])

    # Exporta o relatório em Excel
    with pd.ExcelWriter("relatorio_monitoramento.xlsx", engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name="Detalhes de Alocação", index=False)
        summary_df.to_excel(writer, sheet_name="Resumo", index=False)

    print("Relatório gerado com sucesso: relatorio_monitoramento.xlsx")

# Função para verificar o histórico de criticidade
def check_criticidade_history(criticidade):
    if not os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, "w") as file:
            file.write(f"{criticidade}\n")
        return False

    # Leitura do histórico
    with open(HISTORY_FILE, "r") as file:
        history = file.readlines()

    # Adicionar a criticidade do dia atual
    history.append(f"{criticidade}\n")

    # Manter apenas os últimos 3 dias no histórico
    if len(history) > 3:
        history = history[-3:]

    # Salvar o histórico atualizado
    with open(HISTORY_FILE, "w") as file:
        file.writelines(history)

    # Verifica se os últimos 3 dias foram "Média"
    if history.count("Média\n") >= 3:
        return True
    return False

# Criação de indivíduos e população
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=NUM_AREAS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Funções de avaliação, cruzamento e mutação
toolbox.register("evaluate", eval_allocation)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    population = toolbox.population(n=50)

    # Algoritmo Genético
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=False)

    # Avaliação da melhor solução
    best_ind = tools.selBest(population, 1)[0]
    best_fitness = best_ind.fitness.values[0]
        # Cálculo de custo e capacidade total
    total_cost = sum([COST[i] for i in range(NUM_AREAS) if best_ind[i] == 1])
    total_capacity = sum([CAPACITY[i] for i in range(NUM_AREAS) if best_ind[i] == 1])

    # Classificação da criticidade global
    criticidade = classify_criticidade_cvss(best_fitness)
    print(f"Melhor alocação de sensores: {best_ind}")
    print(f"Criticidade da melhor solução: {criticidade}")
    print(f"Custo total: {total_cost}")
    print(f"Capacidade total: {total_capacity}")

    # Geração do relatório
    generate_report(best_ind, best_fitness, criticidade, total_cost, total_capacity)

    # Verifica o histórico de criticidade e envia alerta se necessário
    if check_criticidade_history(criticidade):
        email_subject = "Alerta de Criticidade Média - 3 Dias Consecutivos"
        email_body = f"""
        A criticidade da alocação de sensores permaneceu em nível 'Média' por 3 dias consecutivos.

        Melhor alocação de sensores: {best_ind}
        Criticidade: {criticidade}
        Fitness: {best_fitness}
        Custo Total: {total_cost}
        Capacidade Total: {total_capacity}
        """
        send_email_alert(email_subject, email_body)

    # Enviar e-mail de alerta para criticidade alta ou crítica
    if criticidade in ["Alta", "Crítica"]:
        email_subject = "Alocação de Sensores - Alerta de Segurança"
        email_body = f"""
        Melhor alocação de sensores foi encontrada:
        Alocação: {best_ind}
        Criticidade: {criticidade}
        Fitness: {best_fitness}
        Custo Total: {total_cost}
        Capacidade Total: {total_capacity}
        """
        send_email_alert(email_subject, email_body)

if __name__ == "__main__":
    main()


Melhor alocação de sensores: [1, 1, 1, 0, 0]
Criticidade da melhor solução: Crítica
Custo total: 1800
Capacidade total: 400
Relatório gerado com sucesso: relatorio_monitoramento.xlsx
E-mail enviado com sucesso!


Método de Força Bruta

In [ ]:
import itertools

def brute_force_solution():
    best_solution = None
    best_fitness = -1

    # Gera todas as combinações possíveis de alocação de sensores (0 ou 1)
    for combination in itertools.product([0, 1], repeat=NUM_AREAS):
        total_critic = 0
        total_cost = 0
        total_capacity = 0

        for i in range(NUM_AREAS):
            if combination[i] == 1:
                vulnerability_factor = CVSS_SCORES[i] if VULNERABILITIES[i] else 1
                total_critic += CRITICITY_LEVELS[i] * PROB_ATTACK[i] * vulnerability_factor
                total_cost += COST[i]
                total_capacity += CAPACITY[i]

        # Verifica se a solução respeita as restrições de custo e capacidade
        if total_cost <= MAX_COST and total_capacity <= MAX_CAPACITY:
            if total_critic > best_fitness:
                best_fitness = total_critic
                best_solution = combination

    return best_solution, best_fitness

# Chama a função de força bruta para comparação
brute_solution, brute_fitness = brute_force_solution()
print(f"Melhor solução (Força Bruta): {brute_solution}, Fitness: {brute_fitness}")


Melhor solução (Força Bruta): (1, 1, 1, 0, 0), Fitness: 25.659999999999997


Metodo Guloso

In [ ]:
def greedy_solution():
    # Calcula a razão criticidade/custo para cada área
    ratios = [(CRITICITY_LEVELS[i] * PROB_ATTACK[i] / COST[i], i) for i in range(NUM_AREAS)]

    # Ordena as áreas pela razão criticidade/custo em ordem decrescente
    ratios.sort(reverse=True)

    total_critic = 0
    total_cost = 0
    total_capacity = 0
    allocation = [0] * NUM_AREAS  # Inicializa a solução com todos os sensores desativados

    for ratio, i in ratios:
        if total_cost + COST[i] <= MAX_COST and total_capacity + CAPACITY[i] <= MAX_CAPACITY:
            allocation[i] = 1  # Aloca o sensor nessa área
            vulnerability_factor = CVSS_SCORES[i] if VULNERABILITIES[i] else 1
            total_critic += CRITICITY_LEVELS[i] * PROB_ATTACK[i] * vulnerability_factor
            total_cost += COST[i]
            total_capacity += CAPACITY[i]

    return allocation, total_critic

# Chama a função gulosa para comparação
greedy_allocation, greedy_fitness = greedy_solution()
print(f"Melhor solução (Método Guloso): {greedy_allocation}, Fitness: {greedy_fitness}")

Melhor solução (Método Guloso): [1, 1, 1, 0, 0], Fitness: 25.66


Comparação entre método e resultado

In [ ]:
import time
def compare_methods():
    # Algoritmo Genético
    start_time = time.time()
    population = toolbox.population(n=50)
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=False)
    best_ind_genetic = tools.selBest(population, 1)[0]
    best_fitness_genetic = best_ind_genetic.fitness.values[0]
    time_genetic = time.time() - start_time

    # Força Bruta
    start_time = time.time()
    best_solution_brute, best_fitness_brute = brute_force_solution()
    time_brute = time.time() - start_time

    # Método Guloso
    start_time = time.time()
    best_solution_greedy, best_fitness_greedy = greedy_solution()
    time_greedy = time.time() - start_time

    # Exibe os resultados
    print("Comparação de Métodos:")
    print(f"Algoritmo Genético: Melhor Fitness = {best_fitness_genetic}, Tempo = {time_genetic:.4f} segundos")
    print(f"Força Bruta: Melhor Fitness = {best_fitness_brute}, Tempo = {time_brute:.4f} segundos")
    print(f"Método Guloso: Melhor Fitness = {best_fitness_greedy}, Tempo = {time_greedy:.4f} segundos")

# Chama a função de comparação
compare_methods()


Comparação de Métodos:
Algoritmo Genético: Melhor Fitness = 25.659999999999997, Tempo = 0.0433 segundos
Força Bruta: Melhor Fitness = 25.659999999999997, Tempo = 0.0001 segundos
Método Guloso: Melhor Fitness = 25.66, Tempo = 0.0000 segundos
